In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import time, datetime
from arcticdb import Arctic, QueryBuilder
import yfinance as yf
import talib
import sys, os


from data_and_research import ac,connect_with_ArcticDB


Connected to ArcticDB at ../db


In [2]:
ac

Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))

In [5]:
ac

Arctic(config=S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb))